In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2021/train.csv
/kaggle/input/tabular-playground-series-jun-2021/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
ids_train = train.id
ids_test = test.id
train = train.drop(columns = 'id')
test = test.drop(columns = 'id')
test = np.array(test)

In [3]:
train.target = train.target.apply(lambda x:int(x[-1:])-1)

In [4]:
train_label=train.target
train= train.drop(columns = 'target')

In [5]:
train_x,test_x,train_labels,test_labels = train_test_split(train,train_label,test_size=0.2,random_state=42)

In [12]:
train_x = np.array(train_x)
test_x = np.array(test_x)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
train_x = torch.from_numpy(train_x).float()
train_labels = torch.from_numpy(train_labels)
test_x = torch.from_numpy(test_x).float()
test_labels = torch.from_numpy(test_labels)

In [11]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn.functional as F
from torchvision import transforms

In [13]:

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        self.x = x
        self.y = y


    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]
    

train_set = MyDataset(train_x,train_labels)
train_loader = torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True)
test_set = MyDataset(test_x,test_labels)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=64,shuffle=True)

In [16]:
class Mynet(nn.Module):
    def __init__(self):
        super(Mynet,self).__init__()
        self.network= nn.Sequential(
            nn.Linear(75,512),
            nn.Sigmoid(),
            nn.Linear(512,256),
            nn.ReLU(),
            #nn.Dropout(p=0.5),
            nn.Linear(256,128),
            nn.Sigmoid(),
            #nn.Dropout(p=0.5),
            nn.Linear(128,32),
            nn.ReLU(),
            nn.Linear(32,9))
    def forward(self,x):
        return self.network(x)
    
model = Mynet()
optimizer = optim.Adam(model.parameters(),lr = 0.0005)
loss_fn = nn.CrossEntropyLoss()
        

In [17]:
epoch = 5
for i in range(epoch):
    batch=0
    loss1=0
    for x,y in train_loader:
        batch+=1
        if torch.cuda.is_available():
            x,y = x.cuda(),y.cuda()
    
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output,y)
        loss.backward()
        optimizer.step()
        loss1 += loss.item()
    loss1/=batch
    print(f"avg loss in {i+1} epoch is {loss1}")


    batch1=0
    loss2=0
    correct =0
    with torch.no_grad():
        for x,y in test_loader:
          if torch.cuda.is_available():
            x,y = x.cuda(),y.cuda()
          batch1+=1
          pred = model(x)
          loss = loss_fn(output,y)
          loss2+=loss.item()
          correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        loss2/=batch
        correct/=len(test_loader.dataset)
        print(f"avg loss while training is {loss2} and accuracy is {correct*100}")

avg loss in 1 epoch is 1.8024201702594758
avg loss while training is 0.5063734341144561 and accuracy is 35.175
avg loss in 2 epoch is 1.774996980047226
avg loss while training is 0.5023246809959412 and accuracy is 35.425000000000004
avg loss in 3 epoch is 1.7665010908603669
avg loss while training is 0.5097041701793671 and accuracy is 35.685
avg loss in 4 epoch is 1.759474776172638
avg loss while training is 0.5217420343399047 and accuracy is 35.66
avg loss in 5 epoch is 1.753707072353363
avg loss while training is 0.5227172166824341 and accuracy is 35.894999999999996


In [18]:

with torch.no_grad():
    test = torch.tensor(test).float()
    if torch.cuda.is_available():
        test = test.cuda()
    pred = F.softmax(model(test),dim=1)
    print(pred[0])

pred = np.array(pred)
ans = pd.DataFrame(pred,index=ids_test,columns=["Class_"+ str(i+1) for i in range(9)])
ans.to_csv("submission.csv")

tensor([0.0682, 0.4647, 0.1591, 0.0229, 0.0128, 0.1105, 0.0173, 0.0326, 0.1118])


In [19]:
!kaggle competitions submit -c tabular-playground-series-jun-2021 -f submission.csv -m "Message"

Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.7/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.7/site-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
